## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-14-06-14-32 +0000,nypost,Iowa National Guard soldiers ID’d as victims i...,https://nypost.com/2025/12/14/us-news/us-troop...
1,2025-12-14-06-09-24 +0000,nyt,A Brown University Instructor Hid From Gunfire...,https://www.nytimes.com/2025/12/14/us/brown-sh...
2,2025-12-14-05-46-15 +0000,nyt,2 Students at Brown Witnessed School Shootings...,https://www.nytimes.com/2025/12/13/us/brown-st...
3,2025-12-14-05-35-47 +0000,nypost,Powerball jackpot grows to colossal $1.10B aft...,https://nypost.com/2025/12/14/us-news/powerbal...
4,2025-12-14-05-12-24 +0000,wapo,"At least 2 dead, 9 injured in shooting at Brow...",https://www.washingtonpost.com/nation/2025/12/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2373/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
113,trump,22
11,university,17
10,brown,16
182,killed,13
7,syria,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
50,2025-12-13-21-55-38 +0000,nypost,"At least 2 killed, 9 people injured in mass sh...",https://nypost.com/2025/12/13/us-news/active-s...,67
25,2025-12-14-01-20-18 +0000,nypost,Terrified Brown University students describe ...,https://nypost.com/2025/12/13/us-news/terrifie...,66
71,2025-12-13-19-32-58 +0000,nypost,"Trump tells ISIS, ‘we will retaliate’ after ‘t...",https://nypost.com/2025/12/13/us-news/presiden...,61
58,2025-12-13-21-00-41 +0000,nypost,Trump attends annual Army-Navy game after cond...,https://nypost.com/2025/12/13/us-news/trump-at...,60
43,2025-12-13-22-50-16 +0000,cbc,"2 killed, 8 injured in Brown University shooti...",https://www.cbc.ca/news/world/brown-university...,60


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
50,67,2025-12-13-21-55-38 +0000,nypost,"At least 2 killed, 9 people injured in mass sh...",https://nypost.com/2025/12/13/us-news/active-s...
73,53,2025-12-13-19-22-53 +0000,nypost,Belarus frees 123 prisoners including Nobel Pe...,https://nypost.com/2025/12/13/world-news/belar...
77,51,2025-12-13-18-51-13 +0000,cbc,Trump vows 'very serious retaliation' after de...,https://www.cbc.ca/news/world/syria-attack-u-s...
111,37,2025-12-13-15-26-20 +0000,nypost,Israel kills top Hamas commander Raad Saad wit...,https://nypost.com/2025/12/13/world-news/israe...
113,28,2025-12-13-14-48-43 +0000,cbc,"Organizer arrested after fans rip up seats, th...",https://www.cbc.ca/news/world/soccer-fans-mess...
84,27,2025-12-13-18-22-37 +0000,nypost,2 dead in Russia following drone attack as mil...,https://nypost.com/2025/12/13/world-news/2-dea...
22,25,2025-12-14-02-29-00 +0000,wsj,"President Trump, in a WSJ interview, conveyed ...",https://www.wsj.com/politics/elections/trump-i...
128,25,2025-12-13-12-41-13 +0000,nypost,Outgoing NYC Mayor Eric Adams warns ‘dark days...,https://nypost.com/2025/12/13/us-news/outgoing...
158,21,2025-12-13-10-02-06 +0000,nyt,Korea’s English Exam Was So Hard It Prompted a...,https://www.nytimes.com/2025/12/13/world/asia/...
47,21,2025-12-13-22-19-55 +0000,nypost,Engine failure forces Tokyo-bound United Airli...,https://nypost.com/2025/12/13/us-news/engine-f...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
